In [7]:
from math import sin, cos, radians, degrees, atan2
import pandas as pd
from datetime import datetime,timedelta

In [8]:
sun_data_df = pd.read_csv("./sundata.csv")

# 確保時間欄位為 datetime 格式
sun_data_df['DateTime'] = pd.to_datetime(sun_data_df['DateTime'])

# 定義太陽方位角的計算函數

def calculate_solar_azimuth(lat, lon, date_time, altitude_angle):
    # 地點經緯度轉換為弧度
    lat = radians(lat)
    lon = radians(lon)

    # 計算儒略日
    timezone_offset = 8  # 假設為台灣時區
    utc_time = date_time - timedelta(hours=timezone_offset)
    n = date_time.timetuple().tm_yday  # 一年中的第幾天

    # 太陽赤緯計算
    delta = radians(23.45 * sin(radians(360 / 365 * (n - 81))))

    # 時角計算
    local_solar_time = utc_time.hour + utc_time.minute / 60 + utc_time.second / 3600 + lon / 15
    H = radians(15 * (local_solar_time - 12))

    # 方位角計算
    altitude_angle = radians(altitude_angle)
    sin_azimuth = (cos(delta) * sin(H)) / cos(altitude_angle)
    cos_azimuth = (sin(altitude_angle) - sin(lat) * sin(delta)) / (cos(lat) * cos(altitude_angle))
    azimuth = atan2(sin_azimuth, cos_azimuth)

    # 將結果轉換為度數並修正範圍
    azimuth = degrees(azimuth)
    if azimuth < 0:
        azimuth += 360
    return azimuth

# 增加太陽方位角計算
sun_data_df['太陽方位角'] = sun_data_df.apply(
    lambda row: calculate_solar_azimuth(
        row['緯度'], 
        row['經度'], 
        row['DateTime'], 
        row['太陽仰角']
    ), 
    axis=1
)

In [9]:
sun_data_df

,location,經度,緯度,地點名稱,DateTime,方位角,太陽仰角,日射量 (W/m²),太陽方位角
0,1,121.544444,23.899444,頂樓一號,2024-01-01 00:00:00,181.000,-88.366208,0.0,138.515569
1,1,121.544444,23.899444,頂樓一號,2024-01-01 00:10:00,181.000,-86.202505,0.0,137.831949
2,1,121.544444,23.899444,頂樓一號,2024-01-01 00:20:00,181.000,-83.943446,0.0,137.158723
3,1,121.544444,23.899444,頂樓一號,2024-01-01 00:30:00,181.000,-81.666617,0.0,136.493326
4,1,121.544444,23.899444,頂樓一號,2024-01-01 00:40:00,181.000,-79.383910,0.0,135.833303
...,...,...,...,...,...,...,...,...,...
746635,17,121.600000,23.983333,自家倉庫,2024-10-31 23:10:00,185.125,-75.947259,0.0,142.108191
746636,17,121.600000,23.983333,自家倉庫,2024-10-31 23:20:00,185.125,-77.567816,0.0,141.327489
746637,17,121.600000,23.983333,自家倉庫,2024-10-31 23:30:00,185.125,-78.921182,0.0,140.572822
746638,17,121.600000,23.983333,自家倉庫,2024-10-31 23:40:00,185.125,-79.899069,0.0,139.840382


In [10]:
# 定義理論日射量計算函數
def calculate_theoretical_panel_irradiance(solar_irradiance, panel_azimuth, solar_azimuth, altitude_angle):
    """
    計算到達太陽能板的理論日射量
    :param solar_irradiance: 太陽日射量 (W/m²)
    :param panel_azimuth: 太陽能板的方位角 (°)
    :param solar_azimuth: 太陽的方位角 (°)
    :param altitude_angle: 太陽仰角 (°)
    :return: 理論日射量 (W/m²)
    """
    if altitude_angle > 0:  # 太陽必須在地平線以上
        # 計算方位角差異（轉為弧度）
        azimuth_difference = radians(panel_azimuth - solar_azimuth)
        # 理論日射量計算
        panel_irradiance = solar_irradiance * cos(radians(altitude_angle)) * cos(azimuth_difference)
        return max(panel_irradiance, 0)  # 避免負值
    else:
        return 0  # 太陽在地平線以下時無輻射量

# 計算理論日射量
sun_data_df['太陽能板日射量 (W/m²)'] = sun_data_df.apply(
    lambda row: calculate_theoretical_panel_irradiance(
        row['日射量 (W/m²)'],
        row['方位角'],  # 太陽能板的方位角
        row['太陽方位角'],  # 太陽的方位角
        row['太陽仰角']  # 太陽仰角
    ),
    axis=1
)

In [11]:
sun_data_df.to_csv("sunlight_data.csv",index=False)